<a href="https://colab.research.google.com/github/cld0033/Tone_It_Down/blob/main/tess_transformers_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install if not installed; hide output
!pip install fsspec==2024.10.0 #gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
!pip install datasets -q
!pip install transformers -q
!pip install torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
#import relevant libraries
import torch
import datasets
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [ ]:
#load a tokenizer and training model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#load a dataset
dataset = datasets.load_dataset("uhoui/text-tone-classifier")
#https://huggingface.co/datasets/uhoui/text-tone-classifier/viewer/default/train?f%5Bidx%5D%5Bmin%5D=80&f%5Bidx%5D%5Bmax%5D=90 <-- I used this one bc of the dataset() function

#tokenize dataset and prepare it for training
def preprocess_function(examples):
    labels = [label_mapping[label] for label in examples["label"]]
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels
    return  model_inputs

In [ ]:
from collections import defaultdict

# Create a dictionary to store the mapping from string labels to integers
label_mapping = defaultdict(lambda: len(label_mapping) + 1)

# Extract unique labels from the original dataset
unique_labels = set(dataset['train']['label'])  # Using 'dataset' instead of 'filter_dataset'

# Ensure all unique labels are in label_mapping
for label in unique_labels:
    _ = label_mapping[label]  # Accessing the label adds it to the mapping

# Update the 'label' column in the original dataset
def update_label(example):
    example['label'] = label_mapping[example['label']]
    return example

filter_dataset = dataset.map(update_label)

In [ ]:
print(filter_dataset)
print("header: \n", dataset['train'].take(10).to_pandas())
print("header: \n", filter_dataset['train'].take(10).to_pandas())

print(unique_labels)

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'label'],
        num_rows: 491
    })
})
header: 
    idx                                               text           label
0    0  I am absolutely thrilled with the service I re...             joy
1    1     It's frustrating when the meeting starts late!           anger
2    2  The news about the community event has left me...         sadness
3    3  Wow, I didn't expect to see my friends here to...        surprise
4    4  I'm really worried about the upcoming exams. I...           worry
5    5  The concert last night was the best experience...             joy
6    6  The constant noise from the construction site ...       annoyance
7    7  I can't believe how beautifully the sunset loo...             awe
8    8  Such a shame that the project was canceled, I ...  disappointment
9    9  The chocolate cake was a delightful surprise a...        pleasure
header: 
    idx                                               te

In [ ]:
#run preprocess function on dataset
tokenized_dataset = filter_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/491 [00:00<?, ? examples/s]

In [ ]:
#manually created splits
num_samples = len(tokenized_dataset['train'])
train_indices, val_indices = train_test_split(range(num_samples), test_size=0.2, random_state=42)

# Create train and validation datasets using select
train_dataset = tokenized_dataset['train'].select(train_indices)
val_dataset = tokenized_dataset['train'].select(val_indices)

# Create a DatasetDict with separate splits
split_dataset = datasets.DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [ ]:
#verify that split happened
print("unsplit dataset: \n", tokenized_dataset)
print("split dataset: \n", split_dataset)


unsplit dataset: 
 DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 491
    })
})
split dataset: 
 DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['idx', 'text', 'label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 99
    })
})


In [ ]:
#fine tune model using trainer API via hugging face
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


In [ ]:

# Define a custom data collator
def custom_data_collator(data):
    # Convert input_ids and labels to tensors
    input_ids = torch.stack([torch.tensor(x['input_ids'], dtype=torch.long) for x in data])
    attention_mask = torch.stack([torch.tensor(x['attention_mask'], dtype=torch.long) for x in data])
    labels = torch.tensor([x['labels'] for x in data], dtype=torch.long)

    # Return dictionary of tensors
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["validation"],
    data_collator=custom_data_collator,
    compute_metrics=compute_metrics # Pass the function here
)

In [ ]:
trainer.train()

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
#Export fine tune model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")